In [1]:
import pandas as pd

# 1. Load the datasets
center_tract_data = pd.read_csv('PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2024_release.csv')
zip_tract_crosswalk = pd.read_excel('HUD 2020 Zip Code - Census Track Crosswalk.xlsx')
life_expectancy = pd.read_csv('US_A.csv')

In [2]:
measure_category = {
    'Current lack of health insurance among adults aged 18-64 years' : 'Healthcare Access and Quality',
    'Visits to doctor for routine checkup within the past year among adults' : 'Healthcare Access and Quality',
    'Mammography use among women aged 50-74 years' : 'Healthcare Access and Quality',
    'Colorectal cancer screening among adults aged 45–75 years' : 'Healthcare Access and Quality',
    'Visited dentist or dental clinic in the past year among adults' : 'Healthcare Access and Quality',
    'Cholesterol screening among adults' : 'Healthcare Access and Quality',
    'Taking medicine to control high blood pressure among adults with high blood pressure' : 'Healthcare Access and Quality',
    'Received food stamps in the past 12 months among adults' : 'Food Quality',
    'Food insecurity in the past 12 months among adults' : 'Food Quality',
    # 'Housing insecurity in the past 12 months among adults' : 'Economic Stability',
    # 'Utility services shut-off threat in the past 12 months among adults' : 'Economic Stability',
    'Lack of reliable transportation in the past 12 months among adults' : 'Transportation',
    'Feeling socially isolated among adults' : 'socioeconomic',
    'Lack of social and emotional support among adults' : 'socioeconomic',
    'Fair or poor self-rated health status among adults' : 'Education Access and Health Literacy',
    'Frequent mental distress among adults' : 'Education Access and Health Literacy',
    'Cognitive disability among adults' : 'Education Access and Health Literacy',
    'Depression among adults' : 'Education Access and Health Literacy',
    'Current cigarette smoking among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    'Binge drinking among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    'No leisure-time physical activity among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    'Obesity among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    'Short sleep duration among adults' : 'Neighborhood and Built Environment / Health Behaviors'
}

In [3]:
center_tract_data = center_tract_data[center_tract_data['Measure'].isin(measure_category.keys())]
center_tract_data['Category'] = center_tract_data['Measure'].map(measure_category)

In [4]:
df = center_tract_data.pivot_table(
    index = 'LocationID',
    columns = 'Measure',
    values = 'Data_Value',
    aggfunc = 'mean'
).reset_index()

In [5]:
from collections import defaultdict
cat_measures = defaultdict(list)
for measure, cat in measure_category.items():
    cat_measures[cat].append(measure)

for cat,measures in cat_measures.items():
    df[cat] = df[measures].mean(axis=1)

In [9]:
# df

In [7]:
file2_deduplicated = zip_tract_crosswalk.drop_duplicates(subset='TRACT')
# Merge both files on 'location_id'
merged_df = pd.merge(df, file2_deduplicated,left_on='LocationID', right_on='TRACT', how='left')

In [10]:
# merged_df

In [12]:
merged_df.columns

Index(['LocationID', 'Binge drinking among adults',
       'Cholesterol screening among adults',
       'Cognitive disability among adults',
       'Colorectal cancer screening among adults aged 45–75 years',
       'Current cigarette smoking among adults',
       'Current lack of health insurance among adults aged 18-64 years',
       'Depression among adults',
       'Fair or poor self-rated health status among adults',
       'Feeling socially isolated among adults',
       'Food insecurity in the past 12 months among adults',
       'Frequent mental distress among adults',
       'Lack of reliable transportation in the past 12 months among adults',
       'Lack of social and emotional support among adults',
       'Mammography use among women aged 50-74 years',
       'No leisure-time physical activity among adults',
       'Obesity among adults',
       'Received food stamps in the past 12 months among adults',
       'Short sleep duration among adults',
       'Taking medicine to

In [13]:
columns_1 = [ 'Binge drinking among adults',
       'Cholesterol screening among adults',
       'Cognitive disability among adults',
       'Colorectal cancer screening among adults aged 45–75 years',
       'Current cigarette smoking among adults',
       'Current lack of health insurance among adults aged 18-64 years',
       'Depression among adults',
       'Fair or poor self-rated health status among adults',
       'Feeling socially isolated among adults',
       'Food insecurity in the past 12 months among adults',
       'Frequent mental distress among adults',
       'Lack of reliable transportation in the past 12 months among adults',
       'Lack of social and emotional support among adults',
       'Mammography use among women aged 50-74 years',
       'No leisure-time physical activity among adults',
       'Obesity among adults',
       'Received food stamps in the past 12 months among adults',
       'Short sleep duration among adults',
       'Taking medicine to control high blood pressure among adults with high blood pressure',
       'Visited dentist or dental clinic in the past year among adults',
       'Visits to doctor for routine checkup within the past year among adults','ZIP', 'TRACT',
       'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'RES_RATIO', 'BUS_RATIO',
       'OTH_RATIO', 'TOT_RATIO']

In [14]:
DF = merged_df.copy()
DF = DF.drop(columns = columns_1,axis=1)

In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scorecard_scaled = DF.copy()
scorecard_scaled[['Healthcare Access and Quality', 'Food Quality','Transportation', 'socioeconomic','Education Access and Health Literacy','Neighborhood and Built Environment / Health Behaviors']] = scaler.fit_transform(
    DF[['Healthcare Access and Quality', 'Food Quality','Transportation', 'socioeconomic','Education Access and Health Literacy','Neighborhood and Built Environment / Health Behaviors']]
)

In [17]:
X = scorecard_scaled.drop(columns=['LocationID'])

In [32]:
X.isnull().sum()

RES_RATIO    57
BUS_RATIO    57
OTH_RATIO    57
TOT_RATIO    57
ZIP          57
dtype: int64

In [33]:
X = X.fillna(X.median())

In [30]:
# scorecard_scaled

In [21]:
thresholds = X.quantile(0.66)

def flag_issues(row):
    flags = {}
    for col in X.columns:
        flags[col] = '⚠️' if row[col] < thresholds[col] else '✅'
    return pd.Series(flags)

scorecard_scaled[[
    'Healthcare_Flag',
    'Food_Flag',
    'Transportaion_Flag',
    'Socioeconomic_Flag',
    'Education_Flag',
    'Neighborhood_Flag'
]] = X.apply(flag_issues, axis=1)

In [23]:
df_2 = scorecard_scaled.drop(columns = ['Healthcare Access and Quality','Food Quality','Transportation','socioeconomic','Education Access and Health Literacy','Neighborhood and Built Environment / Health Behaviors'],axis=1)

In [24]:
merged_scorecard = pd.merge(df_2, file2_deduplicated,left_on='LocationID', right_on='TRACT', how='left')

In [25]:
merged_scorecard = merged_scorecard.drop(columns = ['TRACT','USPS_ZIP_PREF_CITY','USPS_ZIP_PREF_STATE'],axis=1)

In [26]:
flag_map = {'⚠️': 1, '✅': 0}

sdh_flags = [
    'Healthcare_Flag',
    'Food_Flag',
    'Transportaion_Flag',
    'Socioeconomic_Flag',
    'Education_Flag',
    'Neighborhood_Flag'
]

for col in sdh_flags:
    merged_scorecard[col] = merged_scorecard[col].map(flag_map)

In [29]:
# merged_scorecard

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

features = ['RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO','ZIP']  # ZIP-level input columns
X = merged_scorecard[features]
y = merged_scorecard[sdh_flags]

testing model

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.datasets
from sklearn.metrics import accuracy_score

yy = merged_scorecard['Healthcare_Flag']
x_train,x_test,y_train,y_test = train_test_split(X,yy,test_size = 0.2,random_state= 2)

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

x_test_prediction = model.predict(x_test)#it is the prediction with input xtrain
test_data_accuracy = accuracy_score(y_test,x_test_prediction)
print(f"accuracy with training data is :{test_data_accuracy}")

x_train_prediction = model.predict(x_train)#it is the prediction with input xtrain
train_data_accuracy = accuracy_score(y_train,x_train_prediction)
print(f"accuracy with training data is :{train_data_accuracy}")

GradientBoostingClassifier()

model training

In [36]:
from sklearn.ensemble import RandomForestClassifier

models = {}

for col in sdh_flags:
    target_flag = f'{col}'
    model = RandomForestClassifier(random_state=42)
    model.fit(merged_scorecard[features], merged_scorecard[target_flag])
    models[target_flag] = model

In [37]:
def predict_all_flags(zip_input_dict):
    import numpy as np
    X_input = np.array([zip_input_dict[feat] for feat in features]).reshape(1, -1)
    
    predictions = {}
    for flag, model in models.items():
        predictions[flag] = model.predict(X_input)[0]
    
    return predictions

In [38]:
from sklearn.metrics import accuracy_score

for flag in sdh_flags:
    y_true = merged_scorecard[flag]
    y_pred = models[flag].predict(merged_scorecard[features])
    acc = accuracy_score(y_true, y_pred)
    print(f"{flag} Accuracy: {acc:.3f}")

Healthcare_Flag Accuracy: 0.777
Food_Flag Accuracy: 0.802
Transportaion_Flag Accuracy: 0.802
Socioeconomic_Flag Accuracy: 0.827
Education_Flag Accuracy: 0.771
Neighborhood_Flag Accuracy: 0.770


In [40]:
# test_input = {
#     'RES_RATIO': 0.3,
#     'BUS_RATIO': 0.1,
#     'OTH_RATIO': 0.1,
#     'TOT_RATIO': 0.27,
#     'ZIP': 82401,
#     # add all feature names in the same order as in `features`
# }

# result = predict_all_flags(test_input)
# print(result)

## indicators flag

In [41]:
DF_22 = merged_df.copy()
DF_22 = DF_22.drop(columns=['Healthcare Access and Quality', 'Food Quality','Transportation', 'socioeconomic','Education Access and Health Literacy','Neighborhood and Built Environment / Health Behaviors' ,'ZIP', 'TRACT',
       'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'RES_RATIO', 'BUS_RATIO',
       'OTH_RATIO', 'TOT_RATIO'],axis=1)

In [43]:
# Drop the location column to compute thresholds only on data columns
data_cols = DF_22.columns.drop('LocationID')
thresholds = DF_22[data_cols].quantile(0.66)

# Apply flags: 1 if value < threshold else 0
for col in data_cols:
    DF_22[col + '_Flag'] = (DF_22[col] < thresholds[col]).astype(int)

In [48]:
# DF_22

In [45]:
columns_22 = ['Binge drinking among adults',
       'Cholesterol screening among adults',
       'Cognitive disability among adults',
       'Colorectal cancer screening among adults aged 45–75 years',
       'Current cigarette smoking among adults',
       'Current lack of health insurance among adults aged 18-64 years',
       'Depression among adults',
       'Fair or poor self-rated health status among adults',
       'Feeling socially isolated among adults',
       'Food insecurity in the past 12 months among adults',
       'Frequent mental distress among adults',
       'Lack of reliable transportation in the past 12 months among adults',
       'Lack of social and emotional support among adults',
       'Mammography use among women aged 50-74 years',
       'No leisure-time physical activity among adults',
       'Obesity among adults',
       'Received food stamps in the past 12 months among adults',
       'Short sleep duration among adults',
       'Taking medicine to control high blood pressure among adults with high blood pressure',
       'Visited dentist or dental clinic in the past year among adults',
       'Visits to doctor for routine checkup within the past year among adults'] 

In [46]:
DF_22 = DF_22.drop(columns = columns_22,axis=1)

In [49]:
# DF_22

In [54]:
# Map model-flag names to the human-readable category names used in measure_category
flag_to_category = {
    'Healthcare_Flag'   : 'Healthcare Access and Quality',
    'Food_Flag'     : 'Food Quality',
    'Transportaion_Flag' : 'Transportation',
    'Socioeconomic_Flag': 'socioeconomic',
    'Education_Flag'    : 'Education Access and Health Literacy',
    'Neighborhood_Flag' : 'Neighborhood and Built Environment / Health Behaviors'
}

def generate_scorecard(location_id: int, input_features: dict):
    """
    1. Predict the 5 main category flags for the new data point.
    2. Pull that location’s row from DF_22.
    3. For every main category predicted = 1, return the individual measure flags.
    """
    # 1. predict 1/0 for the 5 main categories
    main_preds = predict_all_flags(input_features)          # {'Healthcare_Flag': 1, ...}

    # 2. row for this location (assumes DF_22 already loaded)
    row = DF_22.loc[DF_22['LocationID'] == location_id].iloc[0]

    # 3. build detailed scorecard
    scorecard = {}
    for flag, pred in main_preds.items():
        if pred == 1:                                       # only if category is flagged
            category = flag_to_category[flag]
            scorecard[category] = {
                measure: int(row[measure + '_Flag'])                  # 1/0 from DF_22
                for measure, cat_name in measure_category.items()
                if cat_name == category
            }
    return scorecard



In [55]:
# -------- Example ----------
# location_id you want to review
loc_id = 1001020100  

# dict with values for every feature in your `features` list
input_dict = {
    'RES_RATIO': 0.3,
    'BUS_RATIO': 0.1,
    'OTH_RATIO': 0.1,
    'TOT_RATIO': 0.27,
    'ZIP': 36067
}

scorecard = generate_scorecard(loc_id, input_dict)
print(scorecard)


{'Healthcare Access and Quality': {'Current lack of health insurance among adults aged 18-64 years': 1, 'Visits to doctor for routine checkup within the past year among adults': 1, 'Mammography use among women aged 50-74 years': 0, 'Colorectal cancer screening among adults aged 45–75 years': 1, 'Visited dentist or dental clinic in the past year among adults': 1, 'Cholesterol screening among adults': 1, 'Taking medicine to control high blood pressure among adults with high blood pressure': 0}, 'Food Quality': {'Received food stamps in the past 12 months among adults': 1, 'Food insecurity in the past 12 months among adults': 1}, 'Transportation': {'Lack of reliable transportation in the past 12 months among adults': 1}, 'socioeconomic': {'Feeling socially isolated among adults': 0, 'Lack of social and emotional support among adults': 1}}


C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

In [56]:
import pandas as pd
from IPython.display import display, HTML

def generate_summary_table(scorecard):
    rows = []

    for category, indicators in scorecard.items():
        triggered_indicators = [
            indicator for indicator, flag in indicators.items() if flag == 1
        ]
        if triggered_indicators:
            rows.append({
                'Determinants': category,
                'Example Indicators': "<ul>" + "".join(f"<li>{i}</li>" for i in triggered_indicators) + "</ul>"
            })

    df_summary = pd.DataFrame(rows)
    return df_summary

def display_summary_html(df_summary):
    styled_html = df_summary.to_html(escape=False, index=False)
    display(HTML(styled_html))


In [57]:
summary_df = generate_summary_table(scorecard)
display_summary_html(summary_df)

Determinants,Example Indicators
Healthcare Access and Quality,Current lack of health insurance among adults aged 18-64 yearsVisits to doctor for routine checkup within the past year among adultsColorectal cancer screening among adults aged 45–75 yearsVisited dentist or dental clinic in the past year among adultsCholesterol screening among adults
Food Quality,Received food stamps in the past 12 months among adultsFood insecurity in the past 12 months among adults
Transportation,Lack of reliable transportation in the past 12 months among adults
socioeconomic,Lack of social and emotional support among adults


## solution 

In [68]:
hospital_df = pd.read_excel('Copy of Hospital_General_Information.xlsx')
homehealth_df = pd.read_excel('Copy of HH_Provider_Jan2024.xlsx')
supplier_df = pd.read_excel('Copy of Medical-Equipment-Suppliers.xlsx')

In [63]:
# homehealth_df

In [66]:
def get_healthcare_solution(zipcode: int) -> str:
    """
    Returns the hospital or home health provider name and phone based on ZIP code.
    Priority: Hospital → Home Health. Returns fallback message if not found.
    """
    # Search in hospital dataset
    match = hospital_df[hospital_df['ZIP Code'] == zipcode]
    if not match.empty:
        name  = match.iloc[0]['Facility Name']
        phone = match.iloc[0]['Telephone Number']
        return f"{name} – {phone}"

    # If not found, search in home health dataset
    match = homehealth_df[homehealth_df['zipcode'] == zipcode]
    if not match.empty:
        name  = match.iloc[0]['Provider Name']
        phone = match.iloc[0]['Telephone Number']
        return f"{name} – {phone}"

    # If neither found
    return "No provider found"


In [67]:
zip_input = 36067
solution = get_healthcare_solution(zip_input)
print(solution)


PRATTVILLE BAPTIST HOSPITAL – (334) 361-4267


In [69]:
merged_zip_place_df = pd.merge(center_tract_data, file2_deduplicated,left_on='LocationID', right_on='TRACT', how='left')

In [118]:
# merged_zip_place_df.head(2)

In [115]:
import numpy as np
import math

def _haversine(lon1, lat1, lon2, lat2):
    """Great-circle distance in kilometers."""
    R = 6371.0  # Earth radius (km)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon  = lon2 - lon1
    dlat  = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def get_nearest_supplier(zipcode: int):
    row = merged_zip_place_df .loc[merged_zip_place_df ['ZIP'] == zipcode]
    if row.empty:
        return None  # zip not in table
    point_str = row.iloc[0]['Geolocation']           # "POINT (-80.25 25.96)"
    lon0, lat0 = map(float, point_str.strip('POINT ()').split())
    
    # ---- 2. distances to suppliers
    lons = supplier_df['longitude'].to_numpy(dtype=float)
    lats = supplier_df['latitude' ].to_numpy(dtype=float)
    dists = _haversine(lon0, lat0, lons, lats)
    
    if len(dists) == 0:
        return None  # no suppliers data
    idx = int(np.argmin(dists))
    
    # ---- 3. build answer
    best = supplier_df.iloc[idx]
    return {
        'supplier_name' : best['practicename'],
        'Address'      : str(best['practiceaddress1']),
        'City'     : (best['practicecity']),
        'distance_km'   : float(dists[idx])
    }


In [116]:
result_3332 = get_nearest_supplier(33030)

In [117]:
result_3332

{'supplier_name': 'WALGREENS #11481',
 'Address': '52 E PALM DR',
 'City': 'FLORIDA CITY',
 'distance_km': 1.592402166533423}

In [87]:
supplier_df[['latitude', 'longitude']].isna().sum()


latitude     0
longitude    0
dtype: int64

In [86]:
supplier_df = supplier_df.dropna(subset=['latitude', 'longitude'])


## web scrapping

In [3]:
!pip install selenium webdriver-manager

^C
  Using cached selenium-4.32.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached selenium-4.32.0-py3-none-any.whl (9.4 MB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Using cached h11-0.16.0-py3-none-any.whl (3


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# import time

# def get_food_centers_by_zip(zipcode):
#     # Update the path below to where you saved chromedriver.exe
#     chromedriver_path = r"C:\Users\DELL\.wdm\drivers\chromedriver\win64\136.0.7103.113\chromedriver-win32\chromedriver.exe"  # <-- Update this!
#     service = Service(executable_path=chromedriver_path)
#     options = webdriver.ChromeOptions()
#     # options.add_argument('--headless')  # Uncomment for headless mode

#     driver = webdriver.Chrome(service=service, options=options)
#     url = f"https://www.mealsonwheelsamerica.org/find-meals-and-services/?pc={zipcode}"
#     driver.get(url)
#     time.sleep(5)  # Wait for results to load

#     centers = []
#     elems = driver.find_elements(By.CSS_SELECTOR, '[class*="result"], [class*="card"], [class*="center"]')
#     for elem in elems:
#         text = elem.text.strip()
#         lines = [line.strip() for line in text.split('\n') if line.strip()]
#         if len(lines) >= 3:
#             centers.append({'name': lines[0], 'address': lines[1], 'number': lines[2]})
#         elif len(lines) == 2:
#             centers.append({'name': lines[0], 'address': lines[1], 'number': ''})

#     driver.quit()
#     return centers

# # Example usage
# results = get_food_centers_by_zip("36067")
# for center in results:
#     print(center)


In [3]:
results

[]

In [20]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# import time

# def get_food_centers_by_zip(zipcode):
#     chromedriver_path = r"C:\Users\DELL\.wdm\drivers\chromedriver\win64\136.0.7103.113\chromedriver-win32\chromedriver.exe"  # Update this path
#     service = Service(executable_path=chromedriver_path)
#     options = webdriver.ChromeOptions()
#     driver = webdriver.Chrome(service=service, options=options)

#     url = f"https://www.mealsonwheelsamerica.org/find-meals-and-services/?pc={zipcode}"
#     driver.get(url)
#     time.sleep(5)  # Wait for page to load results

#     centers = []
#     elems = driver.find_elements(By.CLASS_NAME, "card4")
#     for elem in elems:
#         print("CARD TEXT:", elem.text)
#         # Robust extraction as above
#         try:
#             name = elem.find_element(By.CSS_SELECTOR, "h4.card4-title a").text
#             if not name:
#                 name = elem.find_element(By.CSS_SELECTOR, "h4.card4-title").text
#         except:
#             try:
#                 name = elem.find_element(By.CSS_SELECTOR, "h4.card4-title").text
#             except:
#                 name = ""
#         try:
#             address = elem.find_element(By.CSS_SELECTOR, "p.card4-address a").text
#             if not address:
#                 address = elem.find_element(By.CSS_SELECTOR, "p.card4-address").text
#         except:
#             try:
#                 address = elem.find_element(By.CSS_SELECTOR, "p.card4-address").text
#             except:
#                 address = ""
#         try:
#             number = elem.find_element(By.CSS_SELECTOR, "p.card4-action a span").text
#             if not number:
#                 number = elem.find_element(By.CSS_SELECTOR, "p.card4-action").text
#         except:
#             try:
#                 number = elem.find_element(By.CSS_SELECTOR, "p.card4-action").text
#             except:
#                 number = ""
#         centers.append({'name': name, 'address': address, 'number': number})

#     driver.quit()
#     return centers

# # Example usage:
# results = get_food_centers_by_zip("36067")
# for center in results:
#     print(center)


In [16]:
df

""


In [19]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# def get_food_centers_full(zipcode):
#     chromedriver_path = r"C:\Users\DELL\.wdm\drivers\chromedriver\win64\136.0.7103.113\chromedriver-win32\chromedriver.exe"
#     service = Service(executable_path=chromedriver_path)
#     options = webdriver.ChromeOptions()
#     driver = webdriver.Chrome(service=service, options=options)

#     url = f"https://www.mealsonwheelsamerica.org/find-meals-and-services/?pc={zipcode}"
#     driver.get(url)
#     # Wait for result cards to load
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "card4"))
#     )

#     centers = []
#     cards = driver.find_elements(By.CLASS_NAME, "card4")
#     for card in cards:
#         try:
#             detail_link = card.find_element(By.CSS_SELECTOR, "h4.card4-title a").get_attribute("href")
#         except:
#             continue

#         driver.execute_script("window.open('');")
#         driver.switch_to.window(driver.window_handles[1])
#         driver.get(detail_link)
#         # Wait for footer to load
#         try:
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, ".Footer-business-info-item--name"))
#             )
#         except:
#             driver.close()
#             driver.switch_to.window(driver.window_handles[0])
#             continue

#         try:
#             name = driver.find_element(By.CSS_SELECTOR, ".Footer-business-info-item--name").text
#         except:
#             name = ""
#         try:
#             address_parts = driver.find_elements(By.CSS_SELECTOR, ".Footer-business-info-item:not(.Footer-business-info-item--name):not(.Footer-business-info-item--country):not(.Footer-business-info-item--phone):not(.Footer-business-info-item--email)")
#             address = ", ".join([part.text for part in address_parts if part.text.strip()])
#         except:
#             address = ""
#         try:
#             number = driver.find_element(By.CSS_SELECTOR, ".Footer-business-info-item--phone").text
#         except:
#             number = ""

#         centers.append({'name': name, 'address': address, 'number': number})

#         driver.close()
#         driver.switch_to.window(driver.window_handles[0])

#     driver.quit()
#     return centers

# # Example usage:
# results = get_food_centers_full("36067")
# for center in results:
#     print(center)


## transportation

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

#####################################################################
START_URL      = "https://www.communityresourcefinder.org/"          # ← update
CHROMEDRIVER   = r"C:\Users\DELL\.wdm\drivers\chromedriver\win64\136.0.7103.113\chromedriver-win32\chromedriver.exe"
ZIP_CODES    = ["36066", "36067", "36068"]                  # as many as you like
DELAY        = 12                                           # seconds
#####################################################################

service = webdriver.chrome.service.Service(CHROMEDRIVER)
driver  = webdriver.Chrome(service=service)
wait    = WebDriverWait(driver, DELAY)

results      = []
cookie_closed = False      # so we dismiss it only once

try:
    for z in ZIP_CODES:
        driver.get(START_URL)

        # ── 0. close cookie banner if it’s there ───────────────────
        if not cookie_closed:
            try:
                btn = wait.until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR,
                         "#hs-eu-cookie-confirmation-inner button, "
                         "#hs-eu-cookie-confirmation-inner a"))
                )
                btn.click()
                cookie_closed = True
            except:
                pass   # banner didn’t appear this time

        # ── 1. “Community Services” button ─────────────────────────
        wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(.,'Community Services')]")
            )
        ).click()

        # ── 2. choose “Transportation” ─────────────────────────────
        trans_label = wait.until(
            EC.visibility_of_element_located(
                (By.CSS_SELECTOR, "label[for='22-radio_64']")
            )
        )
        # scroll so it’s not hidden behind a sticky header/footer
        driver.execute_script(
            "arguments[0].scrollIntoView({block: 'center'});", trans_label
        )
        # click with JS to avoid interception
        driver.execute_script("arguments[0].click();", trans_label)

        # ── 3. search by ZIP ───────────────────────────────────────
        inp = wait.until(EC.visibility_of_element_located((By.ID, "locationId4")))
        inp.clear()
        inp.send_keys(z, Keys.RETURN)

        # ── 4. wait for at least one result card ───────────────────
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.careseeker-result")
            )
        )

        # ── 5. scrape cards ────────────────────────────────────────
        for card in driver.find_elements(By.CSS_SELECTOR, "div.careseeker-result"):
            try:
                name = card.find_element(By.CSS_SELECTOR, "div.ibox-title a").text.strip()
            except:
                name = ""

            try:
                address = card.find_element(
                    By.CSS_SELECTOR, "input[name='Address']"
                ).get_attribute("value").strip()
            except:
                try:
                    address = card.find_element(
                        By.CSS_SELECTOR, "p.address, div.address"
                    ).text.strip()
                except:
                    address = ""

            try:
                phone = card.find_element(
                    By.CSS_SELECTOR, "a[href^='tel'], p.phone, div.phone, span.tel"
                ).text.strip()
            except:
                phone = ""

            results.append(
                {"search_zip": z, "name": name, "address": address, "phone": phone}
            )

        time.sleep(0.3)   # polite gap before next ZIP

finally:
    driver.quit()

# ── 6. save dataframe ─────────────────────────────────────────────
pd.DataFrame(results).to_csv("transportation_services_2.csv", index=False)
print(f"Done – {len(results)} rows written to transportation_services.csv")


Done – 30 rows written to transportation_services.csv


In [3]:
# results

## socioeconomic

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

#####################################################################
START_URL      = "https://www.communityresourcefinder.org/"          # ← update
CHROMEDRIVER   = r"C:\Users\DELL\.wdm\drivers\chromedriver\win64\136.0.7103.113\chromedriver-win32\chromedriver.exe"
ZIP_CODES    = ["36066", "36067", "36068"]                  # as many as you like
DELAY        = 12                                           # seconds
#####################################################################

service = webdriver.chrome.service.Service(CHROMEDRIVER)
driver  = webdriver.Chrome(service=service)
wait    = WebDriverWait(driver, DELAY)

results_2      = []
cookie_closed = False      # so we dismiss it only once

try:
    for z in ZIP_CODES:
        driver.get(START_URL)

        # ── 0. close cookie banner if it’s there ───────────────────
        if not cookie_closed:
            try:
                btn = wait.until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR,
                         "#hs-eu-cookie-confirmation-inner button, "
                         "#hs-eu-cookie-confirmation-inner a"))
                )
                btn.click()
                cookie_closed = True
            except:
                pass   # banner didn’t appear this time

        # ── 1. “Community Services” button ─────────────────────────
        wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(.,'Community Services')]")
            )
        ).click()

        # ── 2. choose “Transportation” ─────────────────────────────
        trans_label = wait.until(
            EC.visibility_of_element_located(
                (By.CSS_SELECTOR, "label[for='19-radio_1']")
            )
        )
        # scroll so it’s not hidden behind a sticky header/footer
        driver.execute_script(
            "arguments[0].scrollIntoView({block: 'center'});", trans_label
        )
        # click with JS to avoid interception
        driver.execute_script("arguments[0].click();", trans_label)

        # ── 3. search by ZIP ───────────────────────────────────────
        inp = wait.until(EC.visibility_of_element_located((By.ID, "locationId4")))
        inp.clear()
        inp.send_keys(z, Keys.RETURN)

        # ── 4. wait for at least one result card ───────────────────
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.careseeker-result")
            )
        )

        # ── 5. scrape cards ────────────────────────────────────────
        for card in driver.find_elements(By.CSS_SELECTOR, "div.careseeker-result"):
            try:
                name = card.find_element(By.CSS_SELECTOR, "div.ibox-title a").text.strip()
            except:
                name = ""

            try:
                address = card.find_element(
                    By.CSS_SELECTOR, "input[name='Address']"
                ).get_attribute("value").strip()
            except:
                try:
                    address = card.find_element(
                        By.CSS_SELECTOR, "p.address, div.address"
                    ).text.strip()
                except:
                    address = ""

            try:
                phone = card.find_element(
                    By.CSS_SELECTOR, "a[href^='tel'], p.phone, div.phone, span.tel"
                ).text.strip()
            except:
                phone = ""

            results_2.append(
                {"search_zip": z, "name": name, "address": address, "phone": phone}
            )

        time.sleep(0.3)   # polite gap before next ZIP

finally:
    driver.quit()

# ── 6. save dataframe ─────────────────────────────────────────────
pd.DataFrame(results_2).to_csv("Socioeconomic_2.csv", index=False)
print(f"Done – {len(results_2)} rows written to Socioeconomic_services.csv")


Done – 30 rows written to Socioeconomic_services.csv


In [6]:
# results_2

## Education Access

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

#####################################################################
START_URL      = "https://www.communityresourcefinder.org/"          # ← update
CHROMEDRIVER   = r"C:\Users\DELL\.wdm\drivers\chromedriver\win64\136.0.7103.113\chromedriver-win32\chromedriver.exe"
ZIP_CODES    = ["36066", "36067", "36068"]                  # as many as you like
DELAY        = 12                                           # seconds
#####################################################################

service = webdriver.chrome.service.Service(CHROMEDRIVER)
driver  = webdriver.Chrome(service=service)
wait    = WebDriverWait(driver, DELAY)

results_3      = []
cookie_closed = False      # so we dismiss it only once

try:
    for z in ZIP_CODES:
        driver.get(START_URL)

        # ── 0. close cookie banner if it’s there ───────────────────
        if not cookie_closed:
            try:
                btn = wait.until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR,
                         "#hs-eu-cookie-confirmation-inner button, "
                         "#hs-eu-cookie-confirmation-inner a"))
                )
                btn.click()
                cookie_closed = True
            except:
                pass   # banner didn’t appear this time

        # ── 1. “Community Services” button ─────────────────────────
        wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(.,'Community Services')]")
            )
        ).click()

        # ── 2. choose “Transportation” ─────────────────────────────
        trans_label = wait.until(
            EC.visibility_of_element_located(
                (By.CSS_SELECTOR, "label[for='16-radio_7']")
            )
        )
        # scroll so it’s not hidden behind a sticky header/footer
        driver.execute_script(
            "arguments[0].scrollIntoView({block: 'center'});", trans_label
        )
        # click with JS to avoid interception
        driver.execute_script("arguments[0].click();", trans_label)

        # ── 3. search by ZIP ───────────────────────────────────────
        inp = wait.until(EC.visibility_of_element_located((By.ID, "locationId4")))
        inp.clear()
        inp.send_keys(z, Keys.RETURN)

        # ── 4. wait for at least one result card ───────────────────
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.careseeker-result")
            )
        )

        # ── 5. scrape cards ────────────────────────────────────────
        for card in driver.find_elements(By.CSS_SELECTOR, "div.careseeker-result"):
            try:
                name = card.find_element(By.CSS_SELECTOR, "div.ibox-title a").text.strip()
            except:
                name = ""

            try:
                address = card.find_element(
                    By.CSS_SELECTOR, "input[name='Address']"
                ).get_attribute("value").strip()
            except:
                try:
                    address = card.find_element(
                        By.CSS_SELECTOR, "p.address, div.address"
                    ).text.strip()
                except:
                    address = ""

            try:
                phone = card.find_element(
                    By.CSS_SELECTOR, "a[href^='tel'], p.phone, div.phone, span.tel"
                ).text.strip()
            except:
                phone = ""

            results_3.append(
                {"search_zip": z, "name": name, "address": address, "phone": phone}
            )

        time.sleep(0.3)   # polite gap before next ZIP

finally:
    driver.quit()

# ── 6. save dataframe ─────────────────────────────────────────────
pd.DataFrame(results_3).to_csv("Education.csv", index=False)
print(f"Done – {len(results_3)} rows written to Education_services.csv")


Done – 7 rows written to Education_services.csv


In [3]:
# results_3

In [11]:
def get_details_by_zip(zipcode, df):
    match = df[df['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'
    else:
        return "No details found for the given zipcode."

In [16]:
testttt = pd.read_csv("Education.csv")

In [17]:
zipcode_input = 36068
print(get_details_by_zip(zipcode_input, testttt))

name = "Central Alabama Aging Consortium"
address = "818 S Perry St   Montgomery AL 36104-5074"
number = "(334) 240-4666"


In [1]:
def get_trans(zipcode):
    testttt_1 = pd.read_csv("transportation_services_2.csv")
    match = testttt_1[testttt_1['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'

In [2]:
def get_socio(zipcode):
    testttt_2 = pd.read_csv("socioeconomic_2.csv")
    match = testttt_2[testttt_2['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'

In [3]:
def get_edu(zipcode):
    testttt_3 = pd.read_csv("Education.csv")
    match = testttt_3[testttt_3['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'

In [4]:
def get_food(zipcode):
    testttt_3 = pd.read_csv("food_2.csv")
    match = testttt_4[testttt_4['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'

In [5]:
import pandas as pd
from IPython.display import HTML, display

# ── 1.  MAP EACH DETERMINANT TO ITS SOLUTION FUNCTION ──────────────────────────
#    Replace A-E with your real functions.
solution_funcs = {
    "Healthcare Access and Quality":      [get_healthcare_solution, get_nearest_supplier],   # Two functions for Liquidity
    "Food Quality":  [get_food],
    "Transportation":     [get_trans],
    "Socioeconomic":       [get_socio],
    "Education Access and Health Literacy":    [get_edu],
}


NameError: name 'get_healthcare_solution' is not defined

In [ ]:
# Helper to call functions with correct args
def safe_call(func, triggered, zipcode):
    if len(inspect.signature(func).parameters) == 2:
        return func(triggered, zipcode)
    else:
        return func(zipcode)


# Updated generate_summary_table
def generate_summary_table(scorecard, zipcode, func_map=solution_funcs):
    rows = []

    for category, indicators in scorecard.items():
        triggered = [ind for ind, flag in indicators.items() if flag == 1]
        if not triggered:
            continue

        solutions = []
        for fn in func_map.get(category, []):
            try:
                solutions.append(safe_call(fn, triggered, zipcode))
            except Exception as e:
                solutions.append(f"Error: {e}")

        rows.append({
            "Determinants": category,
            "Example Indicators": "<ul>" + "".join(f"<li>{i}</li>" for i in triggered) + "</ul>",
            "Solution": "<ul>" + "".join(f"<li>{s}</li>" for s in solutions) + "</ul>"
        })

    return pd.DataFrame(rows)


# Display HTML
def display_summary_html(df_summary):
    display(HTML(df_summary.to_html(escape=False, index=False)))
✅ Usage
python
Copy
Edit
input_dict = {
    'RES_RATIO': 0.3,
    'BUS_RATIO': 0.1,
    'OTH_RATIO': 0.1,
    'TOT_RATIO': 0.27,
    'ZIP': 36067
}

scorecard = generate_scorecard(loc_id, input_dict)

summary_df = generate_summary_table(scorecard, input_dict['ZIP'])
display_summary_html(summary_df)